# Get Tag Data

## Create Client

In [1]:
import keyring
from trendminer_interface import TrendMinerClient

url = 'https://cs.trendminer.net/'
client_id = 'wdanielsclient'

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz = "Europe/Brussels"
)

/Users/wouter.daniels/PycharmProjects/tm-python-interface/trendminer_interface/client.py:183: VersionMismatchWarning: This SDK version was tested for use with TrendMiner versions [2024.R1.0 | 2024.R1.1 | 2024.R2.0 | 2024.R2.1] while your TrendMiner version is [2024.R3.1-04]. Some functionality might not work as expected. 
  warnings.warn(


## Extract data from single tag
If we are planning to retrieve the latest data from tags, it is best to send index requests to make sure the data is up to date. This action happens automatically when using the TrendMiner UI.

In [2]:
level = client.tag.from_name("[CS]BA:LEVEL.1")
conc = client.tag.from_name("[CS]BA:CONC.1")
temp = client.tag.from_name("[CS]BA:TEMP.1")
phase = client.tag.from_name("[CS]BA:PHASE.1")

level.index()
conc.index()
temp.index()
phase.index()

### Example 1 - Interpolated data for fixed interval
Interpolated data is the default. The main advantage is that we get equidistant timestamps, which will be the same for every tag, as long as we use the same interval.

In [3]:
import pandas as pd

freq = pd.Timedelta(hours=1)

interval = pd.Interval(
    left=pd.Timestamp("2023-01-01", tz=client.tz), 
    right=pd.Timestamp("2023-01-03", tz=client.tz),
    closed="both",
)

ser = level.get_data(interval, freq=freq)
ser.head()

ts
2023-01-01 00:00:00+01:00    28.735641
2023-01-01 01:00:00+01:00    10.856165
2023-01-01 02:00:00+01:00    14.464235
2023-01-01 03:00:00+01:00    40.634544
2023-01-01 04:00:00+01:00    24.326510
Name: [CS]BA:LEVEL.1, dtype: float64

### Example 2 - Index data for the last 8h

Index data contains the min, max, start and end datapoints for every index block. The size of an index block is dependent on the index resolution. We can retrieve the client resolution and see that it is 1 minute. For our server, there would be up to 4 points stored per minute. Note that this number can always be less than 4 when a point is both the start/end and min/max value, or when the block contains less than 4 raw datapoints in the datasource.

In [4]:
client.resolution

Timedelta('0 days 00:01:00')

As a first step, we need to create a pandas Interval representing the latest 8h. It always makes sense to round the end timestamp of 'live' intervals down to at least the client resolution to get nice, round timestamps.

We can then use the interval to retrieve the latest data. Note that the retrieved data usually has irregular timestamps, since it contains the real datapoints logged in the datasource.

In [5]:
length = pd.Timedelta(hours=8)
freq = pd.Timedelta(minutes=1)
now = pd.Timestamp.now(tz=client.tz).floor(client.resolution)

interval = pd.Interval(
    left=now-length, 
    right=now,
    closed="both",
)

ser = conc.get_index_data(interval=interval)
ser.head()

ts
2025-03-26 02:50:23+01:00    0.000000
2025-03-26 03:00:53+01:00    2.026474
2025-03-26 03:01:23+01:00    1.022871
2025-03-26 03:02:23+01:00    3.090765
2025-03-26 03:02:53+01:00    2.619209
Name: [CS]BA:CONC.1, dtype: float64

### Example 3 - Chart data for a fixed interval
Tag chart data is a subset of the indexed data that tries to visually represent the original data as closely as possible. Like index data, the min, max, start and end datapoints are returned per block, only now we need to pass the number of blocks as a parameter.

In [6]:
ser = temp.get_chart_data(interval, periods=10)
ser.head()

ts
2025-03-26 02:50:00+01:00     0.000000
2025-03-26 03:37:53+01:00    20.610619
2025-03-26 03:38:53+01:00    24.054213
2025-03-26 03:45:53+01:00    45.087185
2025-03-26 04:25:53+01:00     9.661495
Name: [CS]BA:TEMP.1, dtype: float64

## Extract data from multiple tags
An easy way to get data from multiple tags into a single DataFrame directly is to first instantiate a TrendHub view instance. Note that this does not mean this view is saved to the appliance. Note that we can only get interpolated tag data from TrendHub views.

Since a view can have multiple layers, a list of DataFrame is returned, one for each layer.

In [13]:
thv = client.trend.view(
    name="last 8h",
    entries=[level, conc, temp, phase],
    layers=[interval],
    context_interval=interval,
    live=False,
)

df_list = thv.get_data(freq=pd.Timedelta(minutes=1))
df_list[0].head()

[CS]BA:LEVEL.1  [CS]BA:CONC.1  [CS]BA:TEMP.1  \
ts                                                                        
2025-03-26 02:50:00+01:00        2.802551       34.16749            0.0   
2025-03-26 02:51:00+01:00        0.000000        0.00000            0.0   
2025-03-26 02:52:00+01:00        0.000000        0.00000            0.0   
2025-03-26 02:53:00+01:00        0.000000        0.00000            0.0   
2025-03-26 02:54:00+01:00        0.000000        0.00000            0.0   

                          [CS]BA:PHASE.1  
ts                                        
2025-03-26 02:50:00+01:00         Phase1  
2025-03-26 02:51:00+01:00         Phase1  
2025-03-26 02:52:00+01:00         Phase1  
2025-03-26 02:53:00+01:00         Phase1  
2025-03-26 02:54:00+01:00         Phase1